In [1]:
import os
import torch
from torch import optim, nn, utils, Tensor
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset
import numpy as np
torch.manual_seed(0)

/home/exitialium/miniconda3/envs/simpleNNstock/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from numpy import double
device = "cuda" if torch.cuda.is_available() else "cpu"

class SimpleNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(104 * 22, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 20)
        self.fc4 = nn.Linear(20, 5)
        self.fc5 = nn.Linear(5, 2)
        self.soft = nn.Softmax(dim = 0)
    
    def forward(self, x):
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        # x = self.soft(x)
        lossFunc = nn.CrossEntropyLoss()
        if y == "1":
            y = torch.tensor([0,1]).float()
        else:
            y = torch.tensor([1,0]).float()
        print(self.soft(logits), y)
        loss = lossFunc(logits, y)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer


In [3]:
simpleNN = SimpleNN().to(device).double()
print(simpleNN)

In [ ]:
directory = 'processedDataset/'
X = list()
Y = list()
for filename in os.listdir(directory)[0:320]:
    f = os.path.join(directory, filename)
    X.append(torch.load(f))
    if filename.split('-')[2].replace(".pt","") == '1':
        Y.append(torch.tensor([0,1]).float())
    else:
        Y.append(torch.tensor([1,0]).float())

In [ ]:
print(Y)
Y[1] == "1"

['0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '1', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '0', '0',

True

In [ ]:
from numpy import double


class stockDataset(Dataset):
    def __init__(self, X, Y):
        super().__init__()
        self.labels = Y
        self.data = X
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        print(self.labels[index])
        return self.data[index], self.labels[index]

In [ ]:
dataset = stockDataset(X,Y)

In [ ]:
train, test = torch.utils.data.random_split(dataset, [320,0])

In [ ]:
train_loader = torch.utils.data.DataLoader(train, 1, shuffle = True)
# test_loader = torch.utils.data.DataLoader(test, 1, shuffle = True)

In [ ]:
trainer = pl.Trainer(limit_train_batches=100, max_epochs=100)
trainer.fit(model=simpleNN, train_dataloaders=train_loader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type    | Params
---------------------------------
0 | fc1  | Linear  | 9.2 M 
1 | fc2  | Linear  | 50.1 K
2 | fc3  | Linear  | 2.0 K 
3 | fc4  | Linear  | 105   
4 | fc5  | Linear  | 12    
5 | soft | Softmax | 0     
---------------------------------
9.2 M     Trainable params
0         Non-trainable params
9.2 M     Total params
36.819    Total estimated model params size (MB)
/home/exitialium/miniconda3/envs/simpleNNstock/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (40) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/40 [00:00<?, ?it/s] 0
0
1
1
0
1
0
1
('0', '0', '1', '1', '0', '1', '0', '1')
tensor([4374507.6117, -683725.8556], dtype=torch.float64,
       grad_fn=<AddBackward0>) tensor([1., 0.])
Epoch 0:   2%|▎         | 1/40 [00:00<00:04,  8.10it/s, loss=0, v_num=19]0
0
1
0
0
0
0
1
('0', '0', '1', '0', '0', '0', '0', '1')
tensor([3785963.1778,  -34521.7735], dtype=torch.float64,
       grad_fn=<AddBackward0>) tensor([1., 0.])
Epoch 0:   5%|▌         | 2/40 [00:00<00:04,  8.53it/s, loss=0, v_num=19]1
0
0
0
0
1
1
0
('1', '0', '0', '0', '0', '1', '1', '0')
tensor([3403067.2812, -558242.9325], dtype=torch.float64,
       grad_fn=<AddBackward0>) tensor([1., 0.])
Epoch 0:   8%|▊         | 3/40 [00:00<00:04,  8.55it/s, loss=0, v_num=19]1
0
0
0
0
1
0
0
('1', '0', '0', '0', '0', '1', '0', '0')
tensor([25777.5104, 10043.8381], dtype=torch.float64, grad_fn=<AddBackward0>) tensor([1., 0.])
Epoch 0:  10%|█         | 4/40 [00:00<00:04,  8.65it/s, loss=0, v_num=19]0
1
0
0
1
1
0
0
('0

/home/exitialium/miniconda3/envs/simpleNNstock/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [10]:
# load checkpoint
checkpoint = "./lightning_logs/version_13/checkpoints/epoch=4-step=500.ckpt"
simpleNN = SimpleNN.load_from_checkpoint(checkpoint).double()

# choose your trained nn.Module
simpleNN.eval()

SimpleNN(
  (fc1): Linear(in_features=2288, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=20, bias=True)
  (fc4): Linear(in_features=20, out_features=5, bias=True)
  (fc5): Linear(in_features=5, out_features=2, bias=True)
  (soft): Softmax(dim=0)
)

In [11]:
sf = nn.Softmax(dim = 0)
simpleNN(X[0])

tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>)

In [12]:
print(Y[0])

0


In [13]:
for i in range(len(X)):
    print(simpleNN(X[i]), Y[i])

tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 1
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 1
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 1
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 1
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch.float64, grad_fn=<AddBackward0>) 0
tensor([-0.0554,  0.0552], dtype=torch

In [337]:
lossFunc = nn.CrossEntropyLoss()
input = torch.tensor([0.54,0.78])
target = torch.tensor([0,1]).float()
print(input)
print(target)
output = lossFunc(input,target)
print(output)

tensor([0.5400, 0.7800])
tensor([0., 1.])
tensor(0.5803)


In [401]:
simpleNN(torch.randn([104, 22], dtype=float))

tensor([-0.1216,  0.1657], dtype=torch.float64, grad_fn=<AddBackward0>)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import numpy as np

# class Model(nn.Module):
#     """
#     Just one Linear layer
#     """
#     def __init__(self, configs):
#         super(Model, self).__init__()
#         self.seq_len = configs.seq_len
#         self.pred_len = configs.pred_len
#         self.Linear = nn.Linear(self.seq_len, self.pred_len)
#         # Use this line if you want to visualize the weights
#         # self.Linear.weight = nn.Parameter((1/self.seq_len)*torch.ones([self.pred_len,self.seq_len]))

#     def forward(self, x):
#         # x: [Batch, Input length, Channel]
#         x = self.Linear(x.permute(0,2,1)).permute(0,2,1)
#         return x # [Batch, Output length, Channel]